In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

In [2]:
from src.models.modules import *
from src.models.loss import L1_clamp_loss
from dataclasses import dataclass
import torch

@dataclass
class SDFTransformerConfig:
    dim_context: int
    dim_input: int
    num_outputs: int
    dim_output: int
    delta: float = 0.1
    num_inds: int = 32
    dim_hidden: int = 128
    num_heads: int = 4
    ln: bool = False

class SDFTransformer(nn.Module):
    def __init__(self, config: SDFTransformerConfig):
        super(SDFTransformer, self).__init__()
        self.config = config
        self.enc = nn.Sequential(
            ISAB(config.dim_context, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
            ISAB(config.dim_hidden, config.dim_hidden, config.num_heads, config.num_inds, ln=config.ln),
        )
        self.input_proj = nn.Linear(config.dim_input, config.dim_hidden)
        self.cross = MAB(config.dim_hidden, config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln)
        self.dec = nn.Sequential(
            PMA(config.dim_hidden, config.num_heads, config.num_outputs, ln=config.ln),
            nn.GELU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.GELU(),
            SAB(config.dim_hidden, config.dim_hidden, config.num_heads, ln=config.ln),
            nn.GELU(),
        )
        self.regr = nn.Sequential(
            nn.Linear(config.dim_hidden, config.dim_output),
            nn.Tanh()
        )

    def forward(self, context: torch.Tensor, x: torch.Tensor, labels: torch.Tensor = None):
        y = self.enc(context)           # [batch_size, context_size, dim_hidden]
        x = self.input_proj(x)          # [batch_size, num_outputs, dim_hidden]
        x = x.repeat(1, y.shape[1], 1)  # [batch_size, context_size, dim_hidden]
        y = self.cross(x, y)            # [batch_size, context_size, dim_hidden]
        y = self.dec(y)                 # [batch_size, num_outputs, dim_hidden]
        y = self.regr(y)                # [batch_size, num_outputs, dim_output]

        loss = None
        if labels is not None:
            loss = L1_clamp_loss(y, labels, self.config.delta)
        return {'loss': loss, 'logits': y}

torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = SDFTransformerConfig(dim_context=4, dim_input=3, num_outputs=1, dim_output=1)
model = SDFTransformer(config).to(device)
print(device)

cuda


In [3]:
from src.models.dataset import LazySampleDataset
from pathlib import Path

project_dir = Path(os.path.abspath('')).resolve().parent
procesed_dir = project_dir / 'data' / 'processed'

train_files = list(procesed_dir.rglob('*_train.hdf5'))
val_files = list(procesed_dir.rglob('*_val.hdf5'))

train_dataset = LazySampleDataset(train_files)
val_dataset = LazySampleDataset(val_files)

In [4]:
from src.data.load_data import get_results_dir
from datetime import datetime

notebook_name = '2024_11_25_GELU'
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
folder_name = f"{notebook_name}-{current_date}"
result_dir = get_results_dir() / folder_name
result_dir.mkdir(parents=True, exist_ok=True)
print(result_dir)

C:\_prog\vm_shared\attention-sdf\results\2024_11_25_GELU-2024-11-25-21-24-03


In [5]:
from transformers import Trainer, TrainingArguments

batch_size = 40
training_args = TrainingArguments(
    output_dir=result_dir / "results",
    eval_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    logging_dir=result_dir / "logs",
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=3,

    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [6]:
trainer.train()
train_dataset.close()
val_dataset.close()

  0%|          | 0/120000 [00:00<?, ?it/s]

{'loss': 0.0287, 'grad_norm': 1.1499810218811035, 'learning_rate': 4.999583333333333e-05, 'epoch': 0.0}
{'loss': 0.0117, 'grad_norm': 0.5112515091896057, 'learning_rate': 4.999166666666667e-05, 'epoch': 0.0}
{'loss': 0.0094, 'grad_norm': 0.8464980721473694, 'learning_rate': 4.99875e-05, 'epoch': 0.0}
{'loss': 0.0065, 'grad_norm': 0.8307167887687683, 'learning_rate': 4.998333333333334e-05, 'epoch': 0.0}
{'loss': 0.0077, 'grad_norm': 0.8284140229225159, 'learning_rate': 4.997916666666667e-05, 'epoch': 0.0}
{'loss': 0.0057, 'grad_norm': 0.7566661834716797, 'learning_rate': 4.9975e-05, 'epoch': 0.0}
{'loss': 0.0067, 'grad_norm': 0.15606160461902618, 'learning_rate': 4.997083333333333e-05, 'epoch': 0.0}
{'loss': 0.0073, 'grad_norm': 0.37704789638519287, 'learning_rate': 4.996666666666667e-05, 'epoch': 0.01}
{'loss': 0.0068, 'grad_norm': 0.8951647877693176, 'learning_rate': 4.99625e-05, 'epoch': 0.01}
{'loss': 0.0084, 'grad_norm': 0.7492260932922363, 'learning_rate': 4.995833333333333e-05, '

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.003432005411013961, 'eval_runtime': 152.7409, 'eval_samples_per_second': 1309.407, 'eval_steps_per_second': 32.735, 'epoch': 1.0}
{'loss': 0.0039, 'grad_norm': 0.30940699577331543, 'learning_rate': 4.374583333333334e-05, 'epoch': 1.0}
{'loss': 0.0028, 'grad_norm': 0.684413731098175, 'learning_rate': 4.374166666666667e-05, 'epoch': 1.0}
{'loss': 0.0022, 'grad_norm': 0.12109076976776123, 'learning_rate': 4.3737500000000006e-05, 'epoch': 1.0}
{'loss': 0.0034, 'grad_norm': 0.04297434911131859, 'learning_rate': 4.373333333333334e-05, 'epoch': 1.0}
{'loss': 0.0036, 'grad_norm': 0.14081774652004242, 'learning_rate': 4.372916666666667e-05, 'epoch': 1.0}
{'loss': 0.0035, 'grad_norm': 0.2912975251674652, 'learning_rate': 4.3725000000000006e-05, 'epoch': 1.0}
{'loss': 0.0026, 'grad_norm': 0.22252598404884338, 'learning_rate': 4.372083333333333e-05, 'epoch': 1.0}
{'loss': 0.0021, 'grad_norm': 0.3903191387653351, 'learning_rate': 4.371666666666667e-05, 'epoch': 1.01}
{'loss': 0.0033

  0%|          | 0/5000 [00:00<?, ?it/s]

...

{'eval_loss': 0.0020276198629289865, 'eval_runtime': 145.0008, 'eval_samples_per_second': 1379.302, 'eval_steps_per_second': 34.483, 'epoch': 6.0}
{'loss': 0.0016, 'grad_norm': 0.21319754421710968, 'learning_rate': 1.2495833333333335e-05, 'epoch': 6.0}
{'loss': 0.0019, 'grad_norm': 0.0411544069647789, 'learning_rate': 1.2491666666666668e-05, 'epoch': 6.0}
{'loss': 0.0015, 'grad_norm': 0.17512618005275726, 'learning_rate': 1.24875e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.15960294008255005, 'learning_rate': 1.2483333333333335e-05, 'epoch': 6.0}
{'loss': 0.0014, 'grad_norm': 0.31974953413009644, 'learning_rate': 1.2479166666666667e-05, 'epoch': 6.0}
{'loss': 0.0022, 'grad_norm': 0.025656072422862053, 'learning_rate': 1.2475e-05, 'epoch': 6.0}
{'loss': 0.0017, 'grad_norm': 0.08972246199846268, 'learning_rate': 1.2470833333333334e-05, 'epoch': 6.0}
{'loss': 0.002, 'grad_norm': 0.1818646788597107, 'learning_rate': 1.2466666666666667e-05, 'epoch': 6.01}
{'loss': 0.0018, 'grad_norm': 

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.0018922861199826002, 'eval_runtime': 143.8134, 'eval_samples_per_second': 1390.691, 'eval_steps_per_second': 34.767, 'epoch': 7.0}
{'loss': 0.0024, 'grad_norm': 0.3337828814983368, 'learning_rate': 6.245833333333334e-06, 'epoch': 7.0}
{'loss': 0.0016, 'grad_norm': 0.026512928307056427, 'learning_rate': 6.241666666666667e-06, 'epoch': 7.0}
{'loss': 0.0022, 'grad_norm': 0.089364193379879, 'learning_rate': 6.2375e-06, 'epoch': 7.0}
{'loss': 0.0014, 'grad_norm': 0.031227711588144302, 'learning_rate': 6.2333333333333335e-06, 'epoch': 7.0}
{'loss': 0.0023, 'grad_norm': 0.3719083070755005, 'learning_rate': 6.229166666666667e-06, 'epoch': 7.0}
{'loss': 0.0019, 'grad_norm': 0.2345866709947586, 'learning_rate': 6.2250000000000005e-06, 'epoch': 7.0}
{'loss': 0.0017, 'grad_norm': 0.2031651884317398, 'learning_rate': 6.220833333333333e-06, 'epoch': 7.0}
{'loss': 0.0019, 'grad_norm': 0.2316626012325287, 'learning_rate': 6.2166666666666676e-06, 'epoch': 7.01}
{'loss': 0.0015, 'grad_no

  0%|          | 0/5000 [00:00<?, ?it/s]

{'eval_loss': 0.001840274198912084, 'eval_runtime': 142.9946, 'eval_samples_per_second': 1398.654, 'eval_steps_per_second': 34.966, 'epoch': 8.0}
{'train_runtime': 7169.2078, 'train_samples_per_second': 669.53, 'train_steps_per_second': 16.738, 'train_loss': 0.0025679752110891667, 'epoch': 8.0}


In [7]:
import json
from dataclasses import asdict

current_date = datetime.now().strftime("%Y-%m-%d")
model_name = f"{current_date}-model"
config_name = f"{current_date}-config.json"
trainer.save_model(result_dir / model_name)

with open(result_dir / config_name, 'w') as f:
    json.dump(asdict(config), f)

In [8]:
from src.visualization.generate_mesh import generate_mesh
from src.data.load_data import get_data_dir

obj_path = get_data_dir() / 'intermediate' / 'bunny' / 'stanford-bunny.obj'
infered_obj_path = result_dir / f"bunny-{current_date}-generalization.obj"
generate_mesh(model, obj_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]

In [9]:
hdf5_path = get_data_dir() / 'processed' / 'bunny' / 'stanford-bunny_train.hdf5'
infered_obj_path = result_dir / f"{current_date}-train.obj"
generate_mesh(model, hdf5_path, infered_obj_path, device, batch_size, resolution=100)

Processing batches:   0%|          | 0/25000 [00:00<?, ?it/s]